In [1003]:
import sys
print(sys.version)

3.7.12 (default, Oct 13 2021, 06:53:03) 
[Clang 13.0.0 (clang-1300.0.29.3)]


In [1004]:
%load_ext autoreload
%autoreload 0

import os, json
from draftfast import rules
from draftfast.optimize import run
from draftfast.orm import Player
from draftfast.csv_parse import salary_download
from draftfast.settings import OptimizerSettings, CustomRule, PlayerPoolSettings
from draftfast.lineup_constraints import LineupConstraints
import pandas as pd
import numpy as np
from scipy import stats
from spread import get_metabet_spread, get_current_rankings, get_fantasy_def_points_against
from collections import defaultdict

# https://www.pro-football-reference.com/years/2021/fantasy.htm

SALARY_FILE = './data/1_22.csv'
WEEK = 20
MIN_SALARY = 4799
WEIGHTED = True

ACTIVE_FILE = './data/active.csv'
MAX_PLAYED = WEEK - 1
print('ready')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
ready


In [1005]:
# https://www.lineups.com/nfl-team-rankings
ranking_df = get_current_rankings(WEEK, 'MTI0NTQ2MDU3OQ==', True)

def get_abbr(x):
    try:
        if type(x) is not dict:
            x = eval(x)
        return x['team_abbr']
    except Exception as e:
        print(e,x)
        

print(ranking_df.shape)
ranking_df['team'] = ranking_df['nav'].apply(get_abbr)
# DEF_KEYS = ['passing_interceptions_rank'
# ranking_df['def_rank'] = ranking_df.apply(lambda row: np.avg([row[k] for k in DEF_KEYS]))
num_teams = ranking_df.shape[0]


return cached data week 20
(32, 52)


In [1006]:
rankings = {x['team']: x for x in ranking_df.to_dict('records')}
# rankings
teams = set(rankings.keys())
print(teams, len(teams))
rankings['LV'] = rankings['OAK']
rankings['JAC'] = rankings['JAX']
del rankings['OAK']
rankings['PHI']

{'ATL', 'CLE', 'MIA', 'NYJ', 'PIT', 'CIN', 'BUF', 'DAL', 'NO', 'SF', 'PHI', 'DEN', 'MIN', 'CAR', 'TEN', 'HOU', 'IND', 'ARI', 'OAK', 'BAL', 'SEA', 'LAC', 'DET', 'LAR', 'CHI', 'WAS', 'KC', 'NYG', 'JAX', 'TB', 'GB', 'NE'} 32


{'Unnamed: 0': 2,
 'points_def': 385,
 'passing_completions': 409,
 'touchdowns': 47,
 'turnover_rank': 26,
 'rushing_attempts_rank': 18,
 'offensive_yards_rank': 10,
 'first_downs': 358,
 'rushing_touchdowns': 18,
 'offensive_rating_rank': 3,
 'turnover': 16,
 'red_zone_conversions_rank': 27,
 'red_zone_percentage_rank': 29,
 'passing_yards_rank': 11,
 'offensive_plays': 1076,
 'third_down_percentage_rank': 23,
 'nav': "{'team_logo_bordered': 'assets/images/nfl/logos/bordered/philadelphia-eagles-largest.svg', 'team_name': 'Eagles', 'opp_division': 'South', 'opp_logo_bordered': 'assets/images/nfl/logos/bordered/tampa-bay-buccaneers-largest.svg', 'team_rank': '2nd', 'opp_depth_chart_route': '/nfl/depth-charts/tampa-bay-buccaneers', 'team_conference': 'NFC', 'opp_league': 'NFC', 'opp_news_route': '/nfl/news/tampa-bay-buccaneers', 'opp_name_full': 'Tampa Bay Buccaneers', 'status': {}, 'opp_name': 'Buccaneers', 'matchup_route': None, 'matchup_news_route': '/nfl/news/matchups/phi-tb', 'game

In [1007]:

spread_df = get_metabet_spread(WEEK)
favor_map = {}
z_map = {}
z_scores = {}

if 'OverUnder' in spread_df.columns.values:
    points = list(spread_df['OverUnder'])
    zs = stats.zscore(points)
    for i, p in enumerate(points):
        z_scores[p] = zs[i]

# https://madduxsports.com/how-to-read-nfl-odds-lines.html"
for index, row in spread_df.iterrows():
    home, away = row['HomeTeam'], row['AwayTeam']
    favor_map[home] = row['PointSpread']
    favor_map[away] = -row['PointSpread']
    if 'OverUnder' in row:
        z_map[home] = z_scores[row['OverUnder']]
        z_map[away] = z_scores[row['OverUnder']]
    

    
if 'JAX' in favor_map:
    favor_map['JAC'] = favor_map['JAX']
    z_map['JAC'] = z_map['JAX']
    
if 'LVS' in favor_map:
    favor_map['LV'] = favor_map['LVS']
    z_map['LV'] = z_map['LVS']

print('favor_map teams', len(favor_map))
print(favor_map)
z_map

return cached data week 20
favor_map teams 8
{'TEN': -3.5, 'CIN': 3.5, 'GB': -5.52, 'SF': 5.52, 'TB': -2.88, 'LAR': 2.88, 'KC': -1.88, 'BUF': 1.88}


{'TEN': -0.5798747606633747,
 'CIN': -0.5798747606633747,
 'GB': -0.7470779125110988,
 'SF': -0.7470779125110988,
 'TB': -0.391326525601052,
 'LAR': -0.391326525601052,
 'KC': 1.7182791987755255,
 'BUF': 1.7182791987755255}

In [1008]:
df = pd.read_csv(SALARY_FILE, na_values= '')
df = df.fillna(df.median())
print(df.describe())

df['Name'] = df['First Name'] + " " + df['Last Name']
df['Salary/FPPG'] = df['FPPG'] / df['Salary']

all_teams = favor_map.keys()
winning_teams = [x for x in all_teams if favor_map[x] < 0]
ordered_teams = sorted(all_teams, key=lambda x: favor_map[x])
print('ordered_teams', [(x, favor_map[x]) for x in ordered_teams])
# excluded_teams = set([x for x in all_teams if favor_map[x] > -MIN_FAVORED])
# top_teams = winning_teams
team_offset = 1
half_teams = int(len(ordered_teams)/2)
top_teams = ordered_teams#[team_offset:team_offset+int(half_teams)]
print(f"top_teams {top_teams}")
excluded_teams = set(all_teams) - set(top_teams)
print(f"excluded_teams {excluded_teams}")

questionable_players = list(df[(~df['Injury Indicator'].isna()) | (~df['Injury Details'].isna())]['Name'])
low_salary_players = list(df[((df['Salary'] < MIN_SALARY)) & (df['Position'] != 'D')]['Name'])

excluded_players = [*questionable_players, *low_salary_players]
excluded_teams = list(df[(df['Position'] == 'D') & df['Team'].isin(excluded_teams)]['Name'])

if excluded_teams:
    excluded_players.extend(excluded_teams)

# excluded_players = ["Rob Gronkowski"]
print(f"Excluding: {len(excluded_players)}")


custom_adds = []

custom_adds = ['Travis Kelce', 'Clyde Edwards-Helaire', "Emmanuel Sanders", "D'Onta Foreman", "Van Jefferson","Rob Gronkowski", "Aaron Jones", "Giovani Bernard","George Kittle"]#, "Ja'Marr Chase"]
# custom_adds = ['AJ Dillon','Darrel Williams']#, 'Deebo Samuel']

if excluded_players:
    excluded_players.extend(custom_adds)
    
questionable_df = df[(df['Name'].isin(set(questionable_players)))]
    
excluded_df = df[(df['Name'].isin(set(excluded_players)))]
    
df = df[(~df['Name'].isin(set(excluded_players)))]# & (df['FPPG'] > 0) & (df['Played'] > 0)

             FPPG      Played       Salary  Tier  Unnamed: 14  Unnamed: 15
count  253.000000  253.000000   253.000000   0.0          0.0          0.0
mean     4.664871    8.229249  5016.600791   NaN          NaN          NaN
std      5.329601    5.556600  1153.308348   NaN          NaN          NaN
min     -0.666667    1.000000  3400.000000   NaN          NaN          NaN
25%      1.020000    4.000000  4500.000000   NaN          NaN          NaN
50%      2.900000    7.000000  4500.000000   NaN          NaN          NaN
75%      6.572222   13.000000  5400.000000   NaN          NaN          NaN
max     25.361111   18.000000  9500.000000   NaN          NaN          NaN
ordered_teams [('GB', -5.52), ('TEN', -3.5), ('TB', -2.88), ('KC', -1.88), ('BUF', 1.88), ('LAR', 2.88), ('CIN', 3.5), ('SF', 5.52)]
top_teams ['GB', 'TEN', 'TB', 'KC', 'BUF', 'LAR', 'CIN', 'SF']
excluded_teams set()
Excluding: 206


In [1009]:
print(winning_teams)

['TEN', 'GB', 'TB', 'KC']


In [1010]:
# http://rotoguru1.com/cgi-bin/fyday.pl?week=1&game=fd&scsv=1

start_week = WEEK-6# take last few games for momentum weighting.

file_names = [f"./history/week{week_number}.csv" for week_number in range(start_week, WEEK+1) if week_number != 18 and os.path.isfile(f"./history/week{week_number}.csv")]
print(file_names, len(file_names))

history_dfs = [pd.read_csv(f, delimiter=";") for f in file_names]
print(f"Using {len(history_dfs)} weeks of history")
historic_data=pd.concat(history_dfs)

if len(historic_data) > 0:
    REPLACE_MAP = {
        'LA': 'Los Angeles',
        '.':'',
    }

    def name_map(x):
        result = ' '.join(x.split(', ')[::-1])
        for k in REPLACE_MAP:
            result = result.replace(k, REPLACE_MAP[k])
        return result


    historic_data['Name'] = historic_data['Name'].apply(name_map)
    team_data = historic_data[historic_data['Pos'] == 'Def']

    historic_data[:1]
    historic_averages = historic_data.groupby("Name").mean()['FD points'].to_dict()
    historic_averages['Patrick Mahomes'] = historic_averages['Patrick Mahomes II']
    historic_averages['Darrell Henderson Jr'] = historic_averages['Darrell Henderson']
    # historic_averages

    team_averages = team_data.groupby("Team").mean()['FD points'].to_dict()
    team_averages['gb'] = team_averages.get('gnb')
    team_averages['kc'] = team_averages.get('kan')
    team_averages['ne'] = team_averages.get('nwe')
    team_averages['tb'] = team_averages.get('tam')
    team_averages['lv'] = team_averages.get('lvr')
    team_averages['no'] = team_averages.get('nor')
    team_averages['sf'] = team_averages.get('sfo')
    # print(team_averages)
    
    for k,v in team_averages.items():
        historic_averages[k] = team_averages[k]
print(len(historic_averages))

['./history/week14.csv', './history/week15.csv', './history/week16.csv', './history/week17.csv'] 4
Using 4 weeks of history
638


In [1011]:
excluded_bonus = defaultdict(lambda: 0)

for index, p in questionable_df.iterrows():
    pos = p['Position']
    if pos in ['TE', 'WR', 'RB', 'QB']:
        points = p['FPPG']
        if points > 7:
            if pos == 'QB':
                # subtract for QB
                amt = -min(points * .08, .8)
            else:
                amt = min(points * .05, .6)
            print('bonus', p['Team'], p['Name'], amt)
            
            excluded_bonus[p['Team']] += amt
        
print(excluded_bonus)

bonus TB Leonard Fournette 0.6
bonus KC Darrel Williams 0.47472220526801223
bonus TB Cyril Grayson 0.6
bonus LAR Darrell Henderson Jr. 0.6
bonus TB Chris Godwin 0.6
bonus TB Antonio Brown 0.6
bonus LAR Robert Woods 0.6
defaultdict(<function <lambda> at 0x16337ed40>, {'TB': 2.4, 'KC': 0.47472220526801223, 'LAR': 1.2})


In [1012]:
df.to_csv(ACTIVE_FILE)
print(f"wrote {ACTIVE_FILE}, {df.shape}")

wrote ./data/active.csv, (77, 19)


In [1013]:
# https://www.fanduel.com/nfl-guide
# Use min/max + roster_size to accomodate flex position. FD roster_size = 9
def get_nfl_positions():
    return [
        ['QB', 1, 1],
        ['RB', 2 ,3],
        ['WR', 3, 4],
        ['TE', 1, 2],
        ['D', 1, 1]
    ]               

In [1014]:
m_score = df.groupby(['Position'])['FPPG'].mean().to_dict()
m_score['DEF'] = m_score['D']
    
m_score

{'D': 7.522058823529412,
 'QB': 7.205102178775054,
 'RB': 10.446835561272609,
 'TE': 6.6170195325215655,
 'WR': 9.654727691791294,
 'DEF': 7.522058823529412}

In [1015]:
max_salary = np.percentile(df[df['Salary'] >= MIN_SALARY]['Salary'], 99)-1
max_salary = 9000
print(max_salary)

9000


In [1016]:
df[df['Position'] == 'D'].shape

(8, 19)

In [1017]:
ACTIVE_RULE_SET = rules.FD_NFL_RULE_SET
# Overrides (position limits, salary, roster size, positions, etc.
ACTIVE_RULE_SET.salary_max = 60000
ACTIVE_RULE_SET.salary_min = ACTIVE_RULE_SET.salary_max - 100
ACTIVE_RULE_SET.defensive_positions = ['D', 'DEF']
ACTIVE_RULE_SET.offensive_positions = ['QB', 'RB', 'WR', 'TE', 'FLEX', 'WR/FLEX']
ACTIVE_RULE_SET.position_limits = get_nfl_positions()
ACTIVE_RULE_SET.max_players_per_team = 9
ACTIVE_RULE_SET.roster_size = 9
print(ACTIVE_RULE_SET.__dict__)

ALL_POSITIONS = [*ACTIVE_RULE_SET.defensive_positions, *ACTIVE_RULE_SET.offensive_positions]

{'site': 'FAN_DUEL', 'league': 'NFL', 'roster_size': 9, 'position_limits': [['QB', 1, 1], ['RB', 2, 3], ['WR', 3, 4], ['TE', 1, 2], ['D', 1, 1]], 'general_position_limits': [], 'salary_min': 59900, 'salary_max': 60000, 'offensive_positions': ['QB', 'RB', 'WR', 'TE', 'FLEX', 'WR/FLEX'], 'defensive_positions': ['D', 'DEF'], 'game_type': 'classic', 'max_players_per_team': 9, 'position_per_team_rules': None, 'min_teams': None}


In [1018]:
# Any additional player or custom rule constraints.
#Player -  https://github.com/BenBrostoff/draftfast/blob/68625902ceea83e66ee9f13a44acd732f600f68f/draftfast/orm.py#L245

# no players min cost (unlikely to play), low score, or favored to lose except overwhelming proj.
# Use salary data from csv as optimization basis.
# set((k, v['defensive_rating_rank']) for k,v in rankings.items())
# rankings['BUF']

allowed_map = get_fantasy_def_points_against(WEEK)
allowed_map

return cached data week 20


{'Jacksonville Jaguars': {'rank': 32, 'allowed': 9.71},
 'Atlanta Falcons': {'rank': 31, 'allowed': 9.59},
 'New York Giants': {'rank': 30, 'allowed': 9.59},
 'Carolina Panthers': {'rank': 29, 'allowed': 9.47},
 'Chicago Bears': {'rank': 28, 'allowed': 9.12},
 'Baltimore Ravens': {'rank': 27, 'allowed': 8.35},
 'New York Jets': {'rank': 26, 'allowed': 8.24},
 'Houston Texans': {'rank': 25, 'allowed': 7.65},
 'Washington Football Team': {'rank': 24, 'allowed': 7.53},
 'Denver Broncos': {'rank': 23, 'allowed': 7.53},
 'Miami Dolphins': {'rank': 22, 'allowed': 7.41},
 'Las Vegas Raiders': {'rank': 21, 'allowed': 6.94},
 'Detroit Lions': {'rank': 20, 'allowed': 6.65},
 'Cleveland Browns': {'rank': 19, 'allowed': 6.53},
 'New Orleans Saints': {'rank': 18, 'allowed': 6.47},
 'Tennessee Titans': {'rank': 17, 'allowed': 6.41},
 'Cincinnati Bengals': {'rank': 16, 'allowed': 6.24},
 'Pittsburgh Steelers': {'rank': 15, 'allowed': 6.06},
 'New England Patriots': {'rank': 14, 'allowed': 5.71},
 'Lo

In [1035]:
players = salary_download.generate_players_from_csvs(salary_file_location=ACTIVE_FILE, game=rules.FAN_DUEL)

FAVOR_DIVISION = 8
AVERAGE_WEIGHT = .65
OVERUNDER_DIVISION = 3

defenses = []
qbs = []

historic_data_used = 0

for p in players:
    p.average_score = m_score[p.pos]
    name = p.name.replace('.', '')
    
    if WEIGHTED:
        base_score = p.proj
        average_score = p.proj
        
        history_key = name_map(p.name) if p.pos != 'D' else p.team.lower()
        history_value = historic_averages.get(history_key)
        if history_value:
            new_score = AVERAGE_WEIGHT*average_score + (1-AVERAGE_WEIGHT)*history_value
            base_score = new_score
            if 'Buffalo' in p.name:
                print(p.name, base_score, average_score, history_value)
            historic_data_used+=1
        else:
            new_score = None
            # print('no historic data', history_key)
            
        
        rank_bonus = 0
        
        # print('rank_bonus', rank_bonus)
        teams = p.matchup.split('@')
        
        
        is_home = p.team == teams[1]
        if is_home:
            rank_bonus += 0.2

        teams.remove(p.team) # remove current players team
        opponent = teams[0]
        
        point_bonus = z_map.get(p.team, 0)
        if point_bonus and p.pos != 'D':
            # max(0, point_bonus/4)
            overunder_bonus = -point_bonus/OVERUNDER_DIVISION if p.pos == 'D' else point_bonus/OVERUNDER_DIVISION # lower points better for defensive scoring.
            rank_bonus += overunder_bonus
            
        # current_rank = rankings[p.team]['overall_rating_rank']
        # opp_rank = rankings[opponent]['overall_rating_rank']
        current_rank = rankings[p.team]['defensive_rating_rank']
        opp_rank = num_teams - rankings[opponent]['offensive_rating_rank']

        if p.pos == 'D':
            opp_bonus = excluded_bonus[opponent]/2
            rank_bonus += opp_bonus
            opp_def_avg = allowed_map[rankings[opponent]['team_fk__full_name']]['allowed']
            base_score = AVERAGE_WEIGHT*base_score + (1-AVERAGE_WEIGHT)*opp_def_avg
        else:
            if p.pos == 'QB':
                # negative (downside for QB injuries)
                injury_bonus = -excluded_bonus[p.team]
            else:
                injury_bonus = excluded_bonus[p.team] 
                
            rank_bonus += injury_bonus
            rank_bonus += -favor_map.get(p.team, 0)/FAVOR_DIVISION
            
        # overall_diff = opp_rank - current_rank # larger the better (should be between 1-32)
        # rank_bonus += overall_diff/num_teams*1.5
        p.proj = base_score + rank_bonus
        
        if p.pos == 'D':
            defenses.append((p.team, average_score, rank_bonus, opp_def_avg, p.proj, p.cost, p.proj / p.cost))
        elif p.pos == 'QB':
            qbs.append((history_key, average_score, base_score, p.proj, p.cost, p.proj / p.cost))
        
        if 'Chase' in name or 'Uzomah' in name:# or p.pos == 'TE':
            print(history_key, history_value, base_score, overunder_bonus, p.proj, p.cost, p.proj / p.cost)
        

print(f"players {len(players)}")
print(f"historic data used {historic_data_used} of {len(players)}") 
# print(players)

for x in sorted(defenses, key=lambda x: x[-1], reverse=True):
    print(x)
# print(excluded_bonus)
print("\n---\n")
for x in sorted(qbs, key=lambda x: x[-1], reverse=True):
    print(x)

Ja'Marr Chase 22.425 18.05013888888889 -0.19329158688779158 17.419347302001096 8200.0 0.0021243106465854995
CJ Uzomah 5.55 6.500146942138673 -0.19329158688779158 5.869355355250882 5500.0 0.0010671555191365239
Buffalo Bills 7.95 9.0 6.0
players 77
historic data used 56 of 77
('BUF', 9.0, 0.23736110263400612, 4.59, 7.011361102634006, 3500.0, 0.0020032460293240018)
('TEN', 7.235294117647059, 0.2, 6.24, 8.114036764705881, 4100.0, 0.001979033357245337)
('KC', 7.5555555555555545, 0.2, 4.12, 7.222972222222222, 3700.0, 0.0019521546546546547)
('LAR', 7.611111111111112, 1.2, 3.41, 7.941069444444445, 4300.0, 0.0018467603359173129)
('TB', 8.333333333333334, 0.8, 5.71, 8.025583333333335, 4500.0, 0.0017834629629629635)
('CIN', 6.4444444444444455, 0.0, 6.41, 5.876277777777779, 3400.0, 0.0017283169934640527)
('SF', 7.0555555555555545, 0.0, 3.53, 6.036472222222222, 3700.0, 0.0016314789789789788)
('GB', 6.941176470588236, 0.2, 5.18, 6.82252205882353, 4500.0, 0.0015161160130718956)

---

('Josh Allen', 2

In [1037]:
# list(filter(lambda p: 'Burrow' in p.name, players))
player_settings = PlayerPoolSettings()
LOCKED = []
BLOCKED_TEAMS = []

# overrides:
# LOCKED = ['Josh Allen', 'Tee Higgins', "Ja'Marr Chase"]#, 'Rob Gronkowski']#, 'C.J. Uzomah']#'Davante Adams']
# LOCKED = ['Eli Mitchell']#, 'Allen Lazard', "Ja'Marr Chase", "Tee Higgins"]
# LOCKED = ['Buffalo Bills','Josh Allen']
# LOCKED = ['Tee Higgins', 'Eli Mitchell', 'Buffalo Bills', 'Tyler Higbee', "Ja'Marr Chase"]
# LOCKED = ['Tee Higgins', 'Eli Mitchell', 'Buffalo Bills', "Ja'Marr Chase", "Josh Allen"]#, 'Tee Higgins']
# BLOCKED_TEAMS = []

constraints = LineupConstraints(locked=LOCKED)

MIN_PROJ = 8.7
min_favored = -6
MIN_LIMIT = -8

roster = None
best_roster = None
best_score = 0


get_score = lambda roster: sum([p.proj for p in roster.players])

while min_favored >= MIN_LIMIT:
    def block_function(p):
        store = p.kv_store
        played = int(float(store.get('Played', -1)))
        
        if 'Single' in p.name:
            return True
        
        should_skip = ((p.team in BLOCKED_TEAMS or p.proj < MIN_PROJ) and p.pos != 'D') or (p.cost > max_salary and p.pos != 'QB') or played < MAX_PLAYED/3 or (p.pos == 'QB' and p.proj < 12)# or (favor_map.get(p.team, 0) > -min_favored)
    
        #print(p.name, played, MAX_PLAYED)
        return should_skip

    opt_settings = OptimizerSettings(
        custom_rules=[
            CustomRule(
                group_a=lambda p: p,
                group_b=block_function, 
                comparison=lambda sum, a, b: sum(b) == 0
            ),
        ],
        min_teams=len(defenses)/2+1
    )

    roster = run(
        rule_set=ACTIVE_RULE_SET,
        player_pool=players,
        verbose=False,
        optimizer_settings=opt_settings,
        constraints=constraints,
        player_settings=player_settings
    )
    
    if roster:
        # total_salary = sum([p.cost for p in roster.players])
        print(roster)
        print('min_favored', min_favored)# negative means players' teams can lose
        print(f"spread weighted: {WEIGHTED}\n\n")
        current_score = get_score(roster)
        if not best_score or current_score > best_score:
            best_score = current_score
            best_roster = roster

    min_favored -= 1
    
    if min_favored >= MIN_LIMIT:
        roster = None
        
if not roster:
    print("No solution")
elif not favor_map:
    print("Warning: No favor map used")

+----------+------------------+------+---------+--------+--------------------+----------+--------+
| Position | Player           | Team | Matchup | Salary |         Projection | vs. Avg. | Locked |
+----------+------------------+------+---------+--------+--------------------+----------+--------+
| QB       | Josh Allen       | BUF  | BUF@KC  |  8,800 | 26.699481955147398 |    17.03 |        |
| RB       | Derrick Henry    | TEN  | CIN@TEN |  9,000 | 23.481708794581937 |    13.03 |        |
| RB       | Sony Michel      | LAR  | LAR@TB  |  5,300 | 10.880391378537713 |     0.43 |        |
| WR       | Davante Adams    | GB   | SF@GB   |  8,700 | 20.529723533252323 |    10.87 |        |
| WR       | Tee Higgins      | CIN  | CIN@TEN |  6,400 | 12.776458677597883 |     3.12 |        |
| WR       | Tyler Boyd       | CIN  | CIN@TEN |  5,800 | 10.259267446674365 |     0.60 |        |
| WR       | Gabriel Davis    | BUF  | BUF@KC  |  5,300 | 10.010780566258507 |     0.36 |        |
| TE      

In [1021]:
# def find_id(first_name, last_name, pos):
#     matches = df.loc[(df['First Name'] == first_name) & (df['Last Name'] == last_name) & (df['Position'] == pos)]
#     if matches:||
#         return matches.iloc[0]['Id']
#     return None
#print(f"{"Player":20}{"Advantage":10}")
if favor_map:
    roster = best_roster
    sorted_players = sorted(roster.players, key=lambda x: favor_map[x.team])
    net_score = 0
    for p in sorted_players:
        advantage = favor_map[p.team]
        name = p.name.replace('.', '')
        print(f"{name:24}{advantage:>10}{historic_averages.get(name, ''):>20} {p.proj} {p.kv_store['Played']} {excluded_bonus[p.team]}")#, p.kv_store['Played'], MAX_PLAYED)
        net_score += advantage
    print(f"---\nTotal adv: {net_score}\n")
    roster.players[-1].__dict__



Davante Adams                -5.52                24.0 20.529723533252323 16.0 0
Derrick Henry                 -3.5                     23.481708794581937 8.0 0
Tennessee Titans              -3.5                     8.114036764705881 17.0 0
Josh Allen                    1.88               28.22 26.699481955147398 18.0 0
Dawson Knox                   1.88                7.35 9.532134732925176 16.0 0
Gabriel Davis                 1.88  13.766666666666666 10.010780566258507 16.0 0
Sony Michel                   2.88               14.45 10.880391378537713 18.0 1.2
Tee Higgins                    3.5              15.575 12.776458677597883 15.0 0
Tyler Boyd                     3.5  13.500000000000002 10.259267446674365 17.0 0
---
Total adv: 2.9999999999999982



In [1022]:
for r in roster.players:
    print(r.team, r.name, r.proj, r.average_score, r.cost,  r.kv_store['Injury Indicator'], r.kv_store['Injury Details'], r.kv_store['Played'])
    # if 'Zay' in r.name:
    #     print(r.__dict__)

TEN Derrick Henry 23.481708794581937 10.446835561272609 9000.0   8.0
BUF Josh Allen 26.699481955147398 7.205102178775054 8800.0   18.0
GB Davante Adams 20.529723533252323 9.654727691791294 8700.0   16.0
BUF Dawson Knox 9.532134732925176 6.6170195325215655 6500.0   16.0
CIN Tee Higgins 12.776458677597883 9.654727691791294 6400.0   15.0
CIN Tyler Boyd 10.259267446674365 9.654727691791294 5800.0   17.0
BUF Gabriel Davis 10.010780566258507 9.654727691791294 5300.0   16.0
LAR Sony Michel 10.880391378537713 10.446835561272609 5300.0   18.0
TEN Tennessee Titans 8.114036764705881 7.522058823529412 4100.0   17.0


In [1023]:
# Create upload CSV by reordering columns to match template order.

headers = []
players = []
ORDERED_COLS = ['QB','RB','RB','WR','WR','WR','TE','FLEX','DEF'] # template order.

def get_match_names(col):
    if col == 'DEF':
        return ['D']
    elif col == 'FLEX' :
        return ['RB', 'WR']
    return [col]

roster_copy = roster.players.copy()
for c in ORDERED_COLS:
    headers.append(c)
    match_names = get_match_names(c)
    for r in roster_copy:
        if r.pos in match_names:
            p = f"{r.kv_store['Id']}:{r.name}"
            players.append(p)
            roster_copy.remove(r)
            break

with open('upload.csv', 'w') as f:
    f.write(','.join(headers))
    f.write('\n')
    f.write(','.join(players))

print('done')


done


In [1024]:
output = pd.read_csv('upload.csv')
output

,QB,RB,RB.1,WR,WR.1,WR.2,TE,FLEX,DEF
0,70496-62239:Josh Allen,70496-39280:Derrick Henry,70496-53640:Sony Michel,70496-45889:Davante Adams,70496-89981:Tee Higgins,70496-40669:Tyler Boyd,70496-64585:Dawson Knox,70496-94534:Gabriel Davis,70496-12534:Tennessee Titans


In [1025]:
m_score = df[(df['FPPG'] > 0) & (df['Played'] > 0)].groupby(['Position'])['FPPG'].mean().to_dict()
m_score['DEF'] = m_score['D']
m_score['FLEX'] = (m_score['WR']+m_score['RB'])/2
m_score

{'D': 7.522058823529412,
 'QB': 9.666802904934064,
 'RB': 10.446835561272609,
 'TE': 6.6170195325215655,
 'WR': 9.654727691791294,
 'DEF': 7.522058823529412,
 'FLEX': 10.05078162653195}

In [1026]:
expected_score = sum([m_score.get(h) for h in headers])
expected_score

83.71451708543607

In [1027]:
# Players that have non-null injury status.
df[(~df['Injury Indicator'].isna()) | ~df['Injury Details'].isna()]

,Id,Position,First Name,Nickname,Last Name,FPPG,Played,Salary,Game,Team,Opponent,Injury Indicator,Injury Details,Tier,Unnamed: 14,Unnamed: 15,Roster Position,Name,Salary/FPPG


In [1028]:
df[df['First Name'] == 'Zay']

,Id,Position,First Name,Nickname,Last Name,FPPG,Played,Salary,Game,Team,Opponent,Injury Indicator,Injury Details,Tier,Unnamed: 14,Unnamed: 15,Roster Position,Name,Salary/FPPG


In [1029]:
# Potentially unaccounted positions
df[~df['Position'].isin(ALL_POSITIONS)]

,Id,Position,First Name,Nickname,Last Name,FPPG,Played,Salary,Game,Team,Opponent,Injury Indicator,Injury Details,Tier,Unnamed: 14,Unnamed: 15,Roster Position,Name,Salary/FPPG


In [1030]:
df[df['Position'] == 'D'][['FPPG', 'Nickname', 'Salary', 'Salary/FPPG']].sort_values('Salary/FPPG', ascending=False)

,FPPG,Nickname,Salary,Salary/FPPG
251,9.000000,Buffalo Bills,3500,0.002571
249,7.555556,Kansas City Chiefs,3700,0.002042
250,7.055556,San Francisco 49ers,3700,0.001907
252,6.444444,Cincinnati Bengals,3400,0.001895
162,8.333333,Tampa Bay Buccaneers,4500,0.001852
202,7.611111,Los Angeles Rams,4300,0.001770
214,7.235294,Tennessee Titans,4100,0.001765
157,6.941176,Green Bay Packers,4500,0.001542


In [1031]:
df[df['Team'] == 'JAC']

,Id,Position,First Name,Nickname,Last Name,FPPG,Played,Salary,Game,Team,Opponent,Injury Indicator,Injury Details,Tier,Unnamed: 14,Unnamed: 15,Roster Position,Name,Salary/FPPG


In [1032]:
spread_df.iloc[1][['HomeTeam', 'PointSpread']]

HomeTeam         GB
PointSpread   -5.52
Name: 1, dtype: object

In [1033]:
from pydfs_lineup_optimizer import Site, Sport, get_optimizer

optimizer = get_optimizer(Site.FANDUEL, Sport.FOOTBALL)
optimizer.load_players_from_csv(ACTIVE_FILE)
for lineup in optimizer.optimize(10):
    print(lineup)


 1. QB      Josh Allen                    QB    BUF            BUF@KC   25.361         8800.0$   
 2. RB      Derrick Henry                 RB    TEN            CIN@TEN  23.038         9000.0$   
 3. RB      Dontrell Hilliard             RB    TEN            CIN@TEN  9.029          5100.0$   
 4. WR      Deebo Samuel                  WR    SF             SF@GB    18.762         8600.0$   
 5. WR      Gabriel Davis                 WR    BUF            BUF@KC   7.469          5300.0$   
 6. WR      Tee Higgins                   WR    CIN            CIN@TEN  12.24          6400.0$   
 7. TE      Tyler Higbee                  TE    LAR            LAR@TB   7.662          5400.0$   
 8. FLEX    Joe Mixon                     RB    CIN            CIN@TEN  16.265         7800.0$   
 9. DEF     Buffalo Bills                 D     BUF            BUF@KC   9.0            3500.0$   

Fantasy Points 128.82
Salary 59900.00

 1. QB      Josh Allen                    QB    BUF            BUF@KC   25.361